# EDA
> EDA란 수집된 데이터에 대해 통계 및 대푯값 추출 등을 통해 다양한 각도에서 관찰하고 이해할 수 있도록 하는 분석 기법을 의미합니다.

## 목차
- import Base code
    - representative_value / 대푯값
    - quantile / 사분위수
    - outlier / 이상치
    - scatterplot / 산점도
    - corr / 상관관계
- 함수 시연
    - representative_value / 대푯값
    - quantile / 사분위수
    - outlier / 이상치
    - scatterplot / 산점도
    - corr / 상관관계

## import Base code


In [1]:
! pip install openpyxl

You should consider upgrading via the '/Users/jheok/Desktop/mobigen/Template-Jupyter-Sample/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import pandas as pd
import numpy as np


file_path = os.path.join(os.getcwd(), "sample_eda.xlsx")


def type_mapping(field_type):
    support = {
        "int64": "INTEGER",
        "float64": "REAL",
        "datetime64": "TIMESTAMP",
        "object": "TEXT",
        np.dtype('float64'): "REAL",
        np.dtype('int32'): "INTEGER",
        np.dtype('int64'): "INTEGER",
        np.dtype('object'): "TEXT"
    }
    python_type = support[field_type]
    return python_type

def parse_df(df):
    if isinstance(df, pd.DataFrame):
        fields = [{"name": a[0], "type": type_mapping(a[1])} for a in zip(df.columns.tolist(), df.dtypes.apply(lambda x: x.name).to_list())]
        return {
            "fields": fields,
            "results": df.values.tolist()
        }
    else:
        return df


## representative_value / 대푯값
> 객체의 기초 통계 정보를 요약해주는 함수입니다. 평균, 표준 편차, 최대/최소값, 사분위수 등을 계산하여 반환합니다.

- 구성
  -  representative_value 실행함수

In [3]:
# representative_value 실행함수

def representative_value(df: pd.DataFrame):
    df = df.describe()
    df = df.reset_index(level=0)
    df.rename(columns={
        'index': '기준열'
    }, inplace=True)
    return df

## quantile / 사분위수
> 4분위수(quartile)란, 데이터를 4개의 집단으로 나누는 것을 의미합니다. 4분위수는 아래와 같은 4가지로 나뉩니다.
  첫번째 사분위수 (Q1) : 데이터의 25%
  두번째 사분위수 (Q2) : 데이터의 50% (이를 중앙값이라고 합니다)
  세번째 사분위수 (Q3) : 데이터의 75%
  네번째 사분위수 (Q4) : 데이터의 100%

- 구성
    - quantile 실행함수

In [4]:
# quantile 실행함수

def rename(newname):
    def decorator(f):
        f.__name__ = newname
        return f

    return decorator


def q_at(target, y):
    @rename(f'{target}_Q{y}')
    def q(x):
        return x.quantile(float(f"0.{y}"))

    return q


def quantile(df: pd.DataFrame):
    by, target, rate = "기준", "값(숫자)", 25
    quantile_list = []
    for percent in range(0, 101, int(rate)):
        quantile_list.append(q_at(target, percent))
    df = df.groupby(by).agg(quantile_list).reset_index(col_level=1)
    df.columns = df.columns.droplevel(level=0)
    df.rename(columns={
        '기준': '기준열', '값(숫자)_Q0': "최솟값", '값(숫자)_Q25': "1사분위",
        '값(숫자)_Q50': '중앙값', '값(숫자)_Q75': '3사분위', '값(숫자)_Q100': '최댓값'
    }, inplace=True)
    return df

## outlier / 이상치
> iqr(interquartile range) 이상치는 통계학에서 사용되는 개념입니다. 이는 데이터의 사분위수(quartile)를 기반으로 하여 이상치를 검출하는 방법입니다.

- 구성
    - outlier 실행함수

In [5]:
# outlier 실행함수

def outlier(df: pd.DataFrame):
    group_by, target = "기준", "값(숫자)"
    target_type = type_mapping(df["값(숫자)"].dtype)

    q3 = df[target].quantile(0.75)
    q1 = df[target].quantile(0.25)
    iqr = q3 - q1

    def iqr_outlier(df):
        if df[target] > q3 + 1.5 * iqr or df[target] < q1 - 1.5 * iqr:
            return df[target]
        else:
            return None

    df['이상치'] = df.apply(iqr_outlier, axis=1)
    df.rename(columns={
        '기준': '범주', '값(숫자)': "값"
    }, inplace=True)
    df = df.replace({np.nan: None})
    res = []
    for i in df.values.tolist():
        tmp = []
        for j in i:
            if j is pd.NA:
                tmp.append(None)
            else:
                tmp.append(j)
        res.append(tmp)
    return {
        "fields": [
            {"name": "범주", "type": "TEXT"},
            {"name": "값", "type": target_type},
            {"name": "이상치", "type": target_type}],
        "results": res,
        "message": None
    }

## scatterplot / 산점도
> DataFrame 객체에서 x, y 열을 지정하여 산점도(Scatter Plot)를 그릴 수 있는 기능입니다.

- 구성
    -  scatterplot 실행함수

In [6]:
# scatterplot 실행함수

def scatterplot(df: pd.DataFrame):
    df.rename(columns={
        '값(숫자)': '값'
    }, inplace=True)
    return df

## corr / 상관관계
>  상관관계 분석은 Pearson 상관계수를 계산합니다.

- 구성
    - corr 실행함수

In [7]:
# corr 실행함수

def corr(df: pd.DataFrame):
    df = df.corr(method='pearson')
    df = df.reset_index(level=0)
    df.rename(columns={
        'index': 'X'
    }, inplace=True)
    df = df.replace({np.nan: None})

    res = []
    for i in df.values.tolist():
        tmp = []
        for j in i:
            if isinstance(j, float):
                tmp.append(round(j, 3))
            else:
                tmp.append(j)
        res.append(tmp)
    return {
        "fields": [{"name": a[0], "type": type_mapping(a[1])}
                   for a in zip(df.columns.tolist(), df.dtypes.apply(lambda x: x.name).to_list())],
        "results": res,
        "message": None
    }

# 함수 시연
## representative_value / 대푯값

In [8]:
df = pd.read_excel(file_path, sheet_name="대푯값")
result = representative_value(df)
print(parse_df(result))

{'fields': [{'name': '기준열', 'type': 'TEXT'}, {'name': '값1(숫자)', 'type': 'REAL'}, {'name': '값2(숫자)', 'type': 'REAL'}, {'name': '값3(숫자)', 'type': 'REAL'}, {'name': '값4(숫자)', 'type': 'REAL'}, {'name': '값5(숫자)', 'type': 'REAL'}], 'results': [['count', 18.0, 18.0, 18.0, 18.0, 18.0], ['mean', 4.994444444444444, 3.4444444444444446, 1.4166666666666665, 0.2277777777777778, 0.2277777777777778], ['std', 0.4193443372726219, 0.41476034500763176, 0.13826657968874304, 0.09582800496696, 0.09582800496696], ['min', 4.3, 2.9, 1.1, 0.1, 0.1], ['25%', 4.725, 3.1, 1.4, 0.2, 0.2], ['50%', 4.95, 3.4, 1.4, 0.2, 0.2], ['75%', 5.325, 3.6750000000000003, 1.5, 0.275, 0.275], ['max', 5.8, 4.4, 1.7, 0.4, 0.4]]}


## quantile / 사분위수

In [9]:
df = pd.read_excel(file_path, sheet_name="사분위수")
result = quantile(df)
print(parse_df(result))

{'fields': [{'name': '기준열', 'type': 'TEXT'}, {'name': '최솟값', 'type': 'REAL'}, {'name': '1사분위', 'type': 'REAL'}, {'name': '중앙값', 'type': 'REAL'}, {'name': '3사분위', 'type': 'REAL'}, {'name': '최댓값', 'type': 'REAL'}], 'results': [['광주', 190.0, 692.25, 1042.5, 1253.25, 397.5], ['나주', 426.0, 1340.25, 1541.0, 1769.5, 875.5], ['대구', 25.0, 188.0, 357.0, 979.75, 78.5], ['대전', 254.0, 576.5, 1187.5, 1605.0, 362.5], ['목포', 589.0, 978.25, 1351.5, 1617.5, 741.0], ['부산', 102.0, 474.5, 1063.0, 1184.25, 205.5], ['서울', 4.0, 139.75, 543.5, 602.25, 9.0], ['시흥', 116.0, 245.75, 382.0, 1120.5, 171.0], ['인천', 886.0, 1049.0, 1382.5, 1704.0, 958.0], ['포항', 761.0, 938.25, 1605.5, 1909.75, 761.5]]}


## outlier / 이상치

In [10]:
df = pd.read_excel(file_path, sheet_name="이상치")
result = outlier(df)
print(result)


{'fields': [{'name': '범주', 'type': 'TEXT'}, {'name': '값', 'type': 'INTEGER'}, {'name': '이상치', 'type': 'INTEGER'}], 'results': [['일', 1598, None], ['월', 1978, None], ['화', 12, 12.0], ['수', 2255, None], ['목', 10000, 10000.0], ['금', 2556, None], ['토', 2387, None]], 'message': None}


## scatterplot / 산점도

In [11]:
df = pd.read_excel(file_path, sheet_name="산점도")
result = scatterplot(df)
print(parse_df(result))

{'fields': [{'name': '범주', 'type': 'INTEGER'}, {'name': '값', 'type': 'INTEGER'}], 'results': [[1807, 114], [2035, 162], [2097, 156], [2342, 185], [2199, 161], [2804, 177], [2857, 179], [1768, 140], [2254, 155], [2359, 144], [2524, 186], [2142, 166], [2511, 166], [2409, 196], [1936, 153], [2931, 215], [2905, 235], [2886, 209]]}


## corr / 상관관계

In [12]:
df = pd.read_excel(file_path, sheet_name="상관관계")
result = corr(df)
print(result)

{'fields': [{'name': 'X', 'type': 'TEXT'}, {'name': '환경', 'type': 'REAL'}, {'name': '기후', 'type': 'REAL'}, {'name': '해양', 'type': 'REAL'}, {'name': '대기', 'type': 'REAL'}], 'results': [['환경', 1.0, 0.181, 0.737, 0.737], ['기후', 0.181, 1.0, 0.318, 0.318], ['해양', 0.737, 0.318, 1.0, 1.0], ['대기', 0.737, 0.318, 1.0, 1.0]], 'message': None}
